In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
#from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import os
from sklearn.model_selection import cross_val_score

In [2]:
df1=pd.read_csv("./processed/training/engaged.csv")
df2=pd.read_csv("./processed/training/notEngaged.csv")
dff=pd.read_csv("./TopFeatures.csv")

In [3]:
dft=pd.concat([df1,df2])

In [4]:
feat=dff.to_numpy()
feat=feat.reshape(-1)

In [5]:
y_train=dft["Result"]
x_train=dft.drop("Result",axis=1)
x_train=x_train[feat]

In [6]:
model = RandomForestClassifier()

In [7]:
avg_score=cross_val_score(model,x_train, y_train, cv=5)

In [8]:
print(avg_score)
print(sum(avg_score)/5)

[0.66787577 0.60598504 0.62321469 0.80934029 0.79913852]
0.7011108592155975


In [9]:
model=model.fit(x_train, y_train)

In [10]:
patho=r'./models'
joblib.dump(model, (os.path.join(patho,r''+'model_joblib_random_forest')))

['./models/model_joblib_random_forest']